In [ ]:
import pandas as pd
import numpy as np
import os

In [78]:
file_path = os.path.join(os.getcwd(), 'data.csv')
columns = ['period', 'TOA', 'GHI', 'BHI', 'DHI', 'BNI']
data_DF = pd.read_csv('data.csv', comment='#', sep=';', header=None, names=columns)
data_DF.head()

,period,TOA,GHI,BHI,DHI,BNI
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0


In [79]:
data_DF[['start', 'end']] = data_DF['period'].str.split('/', expand=True)
# Convert to datetime
data_DF['start'] = pd.to_datetime(data_DF['start'])
data_DF['end'] = pd.to_datetime(data_DF['end'])
data_DF.head()
# data_DF.dtypes
# data_DF.isnull().sum()
# data_DF.duplicated().sum()
# data_DF.describe()

,period,TOA,GHI,BHI,DHI,BNI,start,end
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 00:00:00,2024-04-01 01:00:00
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 01:00:00,2024-04-01 02:00:00
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 02:00:00,2024-04-01 03:00:00
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 03:00:00,2024-04-01 04:00:00
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 04:00:00,2024-04-01 05:00:00


In [ ]:
# Calculate cosine of zenith using 
# GHI = DHI + DNI cos@
# DNI = BNI

data_DF['DNI'] = data_DF['BNI']
cos_zenith = (data_DF['GHI'] - data_DF['DHI']) / data_DF['DNI']

# Clamp values to [-1, 1] to avoid math errors
cos_zenith = np.clip(cos_zenith, -1, 1)
data_DF['cos_zenith'] = cos_zenith

# Compute zenith angle in radians and degree
data_DF['zenith_rad'] = np.arccos(cos_zenith)
data_DF['zenith_deg'] = np.degrees(data_DF['zenith_rad'])
# data_DF.fillna('NA', inplace=True)

# Extraterrestrial irradiance on a normal surface 
# I0n = Isc(1 + 0.033 * cos( 360 * n/ 365))
# Isc​ is the solar constant (1367 W/m²)
# n is the day of the year (from the date in your period column)

data_DF['day_of_year'] = (data_DF['start'] + (data_DF['end'] - data_DF['start']) / 2).dt.dayofyear
data_DF['I0n'] = 1367 * (1 + 0.033 * np.cos(2 * np.pi * data_DF['day_of_year'] / 365))
data_DF['Kt'] = data_DF['GHI'] / (data_DF['I0n'] * data_DF['cos_zenith'])
data_DF['Kd'] = data_DF['DHI'] / data_DF['GHI']
data_DF['Kn'] = data_DF['DNI'] / data_DF['I0n']

# Horizontal extraterrestrial irradiance G0h
data_DF['G0h'] = data_DF['I0n'] * data_DF['cos_zenith']
data_DF.head()

,period,TOA,GHI,BHI,DHI,BNI,start,end,DNI,cos_zenith,zenith_rad,zenith_deg,day_of_year,I0n,Kt,Kd,Kn,G0h
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 00:00:00,2024-04-01 01:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 01:00:00,2024-04-01 02:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 02:00:00,2024-04-01 03:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 03:00:00,2024-04-01 04:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 04:00:00,2024-04-01 05:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN


In [81]:
# K-tests
k_test_condition_1 = (data_DF['GHI'] > 50) & (data_DF['zenith_deg'] < 75) & (data_DF['Kd'] < 1.05)
k_test_condition_2 = (data_DF['GHI'] > 50) & (data_DF['zenith_deg'] > 75) & (data_DF['Kd'] < 1.10)
k_test_condition_3 = (data_DF['Kn'] < 0.8) & (data_DF['Kn'] > 0)
k_test_condition_4 = (data_DF['Kd'] < 0.96) & (data_DF['Kt'] > 0.6)

data_DF['k_test'] = np.where(
    k_test_condition_1 | k_test_condition_2 | k_test_condition_3 | k_test_condition_4,
    'Passed',
    'Failed'
)
data_DF.head()

,period,TOA,GHI,BHI,DHI,BNI,start,end,DNI,cos_zenith,zenith_rad,zenith_deg,day_of_year,I0n,Kt,Kd,Kn,G0h,k_test
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 00:00:00,2024-04-01 01:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 01:00:00,2024-04-01 02:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 02:00:00,2024-04-01 03:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 03:00:00,2024-04-01 04:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 04:00:00,2024-04-01 05:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed


In [82]:
#  Individual Limits Test (ILT)
ILT_condition_1 = (data_DF['GHI'] > -4) & (data_DF['GHI'] < (1.5 * data_DF['I0n'] * data_DF['cos_zenith']**1.2 + 100))
ILT_condition_2 = (data_DF['DHI'] > -4) & (data_DF['DHI'] < (0.97 * data_DF['I0n'] * data_DF['cos_zenith']**1.2 + 50))
ILT_condition_3 = (data_DF['DNI'] > -4) & (data_DF['DNI'] < data_DF['I0n'])
ILT_condition_4 = (data_DF['DHI'] < 0.8 * data_DF['G0h'])
ILT_condition_5 = (data_DF['GHI'] - data_DF['DHI'] < data_DF['G0h'])

data_DF['individual_limit_test'] = np.where(
    ILT_condition_1 | ILT_condition_2 | ILT_condition_3 | ILT_condition_4 | ILT_condition_5,
    'Passed',
    'Failed'
)
data_DF.head()

,period,TOA,GHI,BHI,DHI,BNI,start,end,DNI,cos_zenith,zenith_rad,zenith_deg,day_of_year,I0n,Kt,Kd,Kn,G0h,k_test,individual_limit_test
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 00:00:00,2024-04-01 01:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 01:00:00,2024-04-01 02:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 02:00:00,2024-04-01 03:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 03:00:00,2024-04-01 04:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 04:00:00,2024-04-01 05:00:00,0.0,NaN,NaN,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed


In [83]:
# Night-Time Zero Testing
filter_condition_1 = data_DF['GHI'] > 5
filter_condition_2 = data_DF['zenith_deg'] < 85
data_DF['night_time_test'] = np.where(
    filter_condition_1 | filter_condition_2,
    'Passed',
    'Failed'
)
data_DF.to_csv('output.csv', encoding='utf-8')
data_DF.head()

,period,TOA,GHI,BHI,DHI,BNI,start,end,DNI,cos_zenith,...,zenith_deg,day_of_year,I0n,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2024-04-01T00:00:00.0/2024-04-01T01:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 00:00:00,2024-04-01 01:00:00,0.0,NaN,...,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed,Failed
1,2024-04-01T01:00:00.0/2024-04-01T02:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 01:00:00,2024-04-01 02:00:00,0.0,NaN,...,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed,Failed
2,2024-04-01T02:00:00.0/2024-04-01T03:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 02:00:00,2024-04-01 03:00:00,0.0,NaN,...,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed,Failed
3,2024-04-01T03:00:00.0/2024-04-01T04:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 03:00:00,2024-04-01 04:00:00,0.0,NaN,...,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed,Failed
4,2024-04-01T04:00:00.0/2024-04-01T05:00:00.0,0.0,0.0,0.0,0.0,0.0,2024-04-01 04:00:00,2024-04-01 05:00:00,0.0,NaN,...,NaN,92,1366.417604,NaN,NaN,0.0,NaN,Failed,Passed,Failed
